
 * 119 구급대원 출동 일지 작성 폼 구성
 * 미션3에서 저장된 모델을 불러오기
 * 환자 증상에 따른 중증 질환 예측하기
 * 중증 질환 및 환자 위치에 따른 인근 병원 출력하기

In [ ]:

# 라이브러리 불러오기 

import pandas as pd
import numpy as np
import datetime
import joblib
from keras.models import load_model
from haversine import haversine
from urllib.parse import quote
import streamlit as st
from streamlit_folium import st_folium
import folium
import branca
from geopy.geocoders import Nominatim
import ssl
from urllib.request import urlopen


# -------------------- ▼ 필요 함수 생성 코딩 Start ▼ --------------------


# geocoding : 거리주소 -> 위도/경도 변환 함수
# Nominatim 파라미터 : user_agent = 'South Korea', timeout=None
# 리턴 변수(위도,경도) : lati, long
# 참고: https://m.blog.naver.com/rackhunson/222403071709

In [3]:

!pip install streamlit
!pip install streamlit_folium
!pip install folium
!pip install geopy
!pip install haversine
!pip install joblib


     -------------------------------------- 423.4/423.4 kB 4.4 MB/s eta 0:00:00


## 2) streamlit으로 구현되는 프로토타입 작성
 * 파일명은 dispatch_log.py로 함

In [19]:
%%writefile dispatch_log.py

import pandas as pd
import numpy as np
import datetime
import joblib
from keras.models import load_model
from haversine import haversine
from urllib.parse import quote
import streamlit as st
from streamlit_folium import st_folium
import folium
import branca
from geopy.geocoders import Nominatim
import ssl
from urllib.request import urlopen


def geocoding(address):
    test = Library.loc[0, '새주소명'][:-8]

    geolocoder = Nominatim(user_agent='kbs')
    location = geolocator.geocode(test)
    lati = location.latitude
    long = location.longitude
    return lati, long


# preprocessing : '발열', '고혈압', '저혈압' 조건에 따른 질병 전처리 함수(미션3 참고)
# 리턴 변수(중증질환,증상) : X, Y
def preprocessing(desease):
    
    target = '중증질환'

    desease['발열'] = [ 1 if x >=37 else 0 for x in desease['체온']]
    desease['고혈압'] = [1 if x >= 140 else 0 for x in desease['수축기 혈압']]
    desease['저혈압'] = [1 if x <= 90 else 0 for x in desease['수축기 혈압']]
    
    x = desease[['체온', '수축기 혈압', '이완기 혈압', '호흡 곤란', '간헐성 경련', '설사', '기침', '출혈', '통증', '만지면 아프다','무감각', '마비', '현기증', '졸도', '말이 어눌해졌다', '시력이 흐려짐', '발열', '고혈압', '저혈압']]
    y = desease['중증질환']

    desease['중증질환'] = desease['중증질환'].map({'심근경색':0, '복부손상':1, '뇌경색':2, '뇌출혈':3})
    
    return x, y

# predict_disease : AI 모델 중증질환 예측 함수 (미션1 참고)
# 사전 저장된 모델 파일 필요(119_model_XGC.pkl)
# preprocessing 함수 호출 필요 
# 리턴 변수(4대 중증 예측) : sym_list[pred_y_XGC[0]]

def predict_disease(new_dispatch):
    
    sym_list = ['뇌경색', '뇌출혈', '복부손상', '심근경색']
    
    new_data = pd.DataFrame(new_dispatch)
    new_x, new_y = preprocessing(new_data)
    
    model_m = joblib.load('119_model_xgboost.pkl')
    
    pred_new_m = model_m.predict(new_x)
    
    return sym_list[pred_new_m[0]]


# find_hospital : 실시간 병원 정보 API 데이터 가져오기 (미션1 참고)
# 리턴 변수(거리, 거리구분) : distance_df
def find_hospital(special_m, lati, long):

    context=ssl.create_default_context()
    context.set_ciphers("DEFAULT")
      
    #  [국립중앙의료원 - 전국응급의료기관 조회 서비스] 활용을 위한 개인 일반 인증키(Encoding) 저장
    key = "P3ePdR9kLonvmZAN7dpFadmkVKXmkPKMbmyywKCrmKMxxhNjjEaOr8kmfAHXyISBN%2BB%2FsVTPiDwGbyc2J%2BHM5A%3D%3D"

    # city = 대구광역시, 인코딩 필요
    city = quote("대구광역시")    
    
    # 3) 병원 리스트 csv 파일 불러오기
    solution_df = pd.read_csv('./daegu_hospital_list.csv')

    # 4) 병원 실시간 정보 가져오기

    # 응급실 실시간 가용병상 

    url_realtime = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire' + '?serviceKey=' + key + '&STAGE1='+ city + '&pageNo=1&numOfRows=100'
    result = urlopen(url_realtime, context=context)
    emrRealtime = pd.read_xml(result, xpath='.//item')

    solution_df = pd.merge(solution_df,emrRealtime[['hpid', 'hvec', 'hvoc']] )

    # 실시간 중증질환자 수용 가능 병원 조회
    url_acpt = 'https://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire' + '?serviceKey=' + key + '&STAGE1='+ city + '&pageNo=1&numOfRows=100'
    result = urlopen(url_acpt, context=context)
    emrAcpt = pd.read_xml(result, xpath='.//item')

    emrAcpt = emrAcpt.rename(columns={"dutyName":"hpid"})

    solution_df = pd.merge(solution_df, 
                           emrAcpt[['hpid', 'MKioskTy1', 'MKioskTy2', 'MKioskTy3', 'MKioskTy4', 'MKioskTy5', 'MKioskTy7','MKioskTy8','MKioskTy10', 'MKioskTy11']])

    # 컬럼명 변경하기

    column_change = {  'hpid' : '병원코드',
                       'dutyName' : '병원명',
                       'dutyAddr' : '주소',
                       'dutyTel3' : '응급연락처',
                       'wgs84Lat' : '위도', 
                       'wgs84Lon' : '경도',
                       'hperyn' : '응급실수',
                       'hpopyn' : '수술실수',
                       'hvec' : '가용응급실수',
                       'hvoc' : '가용수술실수',
                       'MKioskTy1' : '뇌출혈', 
                       'MKioskTy2' : '뇌경색',
                       'MKioskTy3' : '심근경색',
                       'MKioskTy4' : '복부손상',
                       'MKioskTy5' : '사지접합',
                       'MKioskTy7' : '응급투석',
                       'MKioskTy8' : '조산산모',
                       'MKioskTy10' : '신생아',
                       'MKioskTy11' : '중증화상',                   
                       }

    solution_df = solution_df.rename(columns=column_change)

    solution_df = solution_df.replace({"정보미제공": "N"})

    # 5) 병원 응급실 가용율, 포화도 계산하기
    ## 가용 응급실수, 가용 수술실 수가 0보다 작은 경우는 비정상 데이터로 추정
    ## 0보다 작은 수는 0으로 변경

    solution_df.loc[solution_df['가용응급실수']<0, '가용응급실수'] = 0
    solution_df.loc[solution_df['가용수술실수']<0, '가용수술실수'] = 0

    # 응급실 가용율 계산 : 가용 응급실수 / 응급실 수
    solution_df['응급실가용율'] = round(solution_df['가용응급실수'] / solution_df['응급실수'], 2)

    # 응급실 가용율이 1이 넘는 경우는 1로 대체
    solution_df.loc[solution_df['응급실가용율'] > 1,'응급실가용율']=1

    # 응급실 가용율에 따라 포화도 분류
    solution_df['응급실포화도'] =  pd.cut(solution_df['응급실가용율'],  bins=[-np.inf, 0.1, 0.3, 0.6, 1], labels=['불가', '혼잡', '보통', '원활'])

    # 6) 환자 수용 가능한 병원의 거리 구하기 (환자 중증 질환 : 응급투석, 환자 위치 : 대구역 근처 (35.8765167, 128.5972922))

    # 중증질환자 수용이 가능하고, 응급실 수용이 가능한 병원
    mkiosk = '응급투석'
    condition1 = (solution_df[mkiosk] == 'Y') & (solution_df['가용수술실수']>=1)
    condition2 = (solution_df['응급실포화도']!= '불가')

    distance_df = solution_df[condition1 & condition2].copy()

    # 거리 계산

    distance = []
    patient = (35.8765167, 128.5972922)

    for idx, row in distance_df.iterrows():
        distance.append(round(haversine((row['위도'], row['경도']), patient, unit = 'km'), 2))

    distance_df['거리'] = distance

    # 7) 거리 구간 구하기

    distance_df['거리구분'] = pd.cut(distance_df['거리'], bins=[-1, 2, 5, 10, np.inf], labels=['2km이내', '5km이내', '10km이내', '10km이상'])
                              
            
    return distance_df

# -------------------- 필요 함수 생성 코딩 END --------------------

data = pd.read_csv('./119_emergency_dispatch_1.csv', encoding="cp949")

## 오늘 날짜
now_date = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=9)
now_date2 = datetime.datetime.strptime(now_date.strftime("%Y-%m-%d"), "%Y-%m-%d")

## 출동 이력의 최소 날짜, 최대 날짜
min_date = datetime.datetime.strptime("2023-01-01", "%Y-%m-%d")
max_date = datetime.datetime.strptime("2023-12-31", "%Y-%m-%d")
today_date = now_date.strftime("%Y-%m-%d")



# 레이아웃 구성하기 
st.set_page_config(layout="wide")

# tabs 만들기 
tab1, tab2 = st.tabs(["환자 정보", "대시보드"])

# tab1 내용물 구성하기 
with tab1:
    
    # 시간 정보 가져오기 
    now_date = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=9)

    
    # 환자정보 널기
    st.markdown("#### 환자 정보")

    ##  ▼ 1-1그룹 날짜/시간 입력 cols 구성(출동일/날짜정보(input_date)/출동시간/시간정보(input_time)) ▼ --------------------
     
    col110, col111, col112, col113 = st.columns([0.1, 0.3, 0.1, 0.3])
    
    with col110:
        st.info("출동일시")
    with col111:
        st.date_input("",label_visibility = 'collapsed')
    with col112:
        st.info("출동시간")
    with col113:
        st.time_input("",label_visibility = 'collapsed')
     
  
        
    ##  ▼ 1-2그룹 이름/성별 입력 cols 구성(이름/이름 텍스트 입력(name)/나이/나이 숫자 입력(age)/성별/성별 라디오(patient_s)) ▼ 

    col120, col121, col122, col123, col124, col125 = st.columns([0.1, 0.3, 0.1, 0.1, 0.1, 0.1])
    
    with col120:
        st.info("이름")
    with col121:
        name = st.text_input("", label_visibility = 'collapsed')
    with col122:
        st.info("나이")
    with col123:
         st.number_input("나이",min_value = 0 , max_value = 120, label_visibility = 'collapsed')                               
    with col124:
        st.info("성별")
    with col125:
         select_raido = st.radio('성별', ['남자','여자'], 
                                 horizontal =  True, label_visibility = 'collapsed')


        
    ## -------------------- ▼ 1-3그룹 체온/환자위치(주소) 입력 cols 구성(체온/체온 숫자 입력(fever)/환자 위치/환자위치 텍스트 입력(location)) ▼ --------------------
    #col 나누기
    col130, col131, col132, col133 = st.columns([0.1, 0.3, 0.1, 0.3])

    with col130:
        st.info("체온")
    with col131:
        fever = st.number_input("",min_value = 0 , max_value = 120, label_visibility = 'collapsed')
    with col132:
        st.info("환자위치")
    with col133:
        location = st.text_input("", key="location_input", label_visibility='collapsed')
        
        
    ##  ▼ 1-4그룹 혈압 입력 cols 구성(수축기혈압/수축기 입력 슬라이더(high_blood)/이완기혈압/이완기 입력 슬라이더(low_blood)) ▼ --------------------
    ## st.slider 사용

    col140, col141, col142, col143 = st.columns([0.1, 0.3, 0.1, 0.3])
    with col140:
        st.info("수축기 혈압")
    with col141:
        high_blood = st.slider('', 0, 200, 140,label_visibility='collapsed') #140이상 고혈압, 90이하 저혈압
    with col142:
        st.info("이완기 혈압")
    with col143:
        low_blood = st.slider('', 0, 200, 80,label_visibility='collapsed') # 90이상 고혈압, 60이하 저혈압
    
    
    
    ##-------------------------------------------------------------------------------------

    ##  ▼ 1-5그룹 환자 증상체크 입력 cols 구성(증상체크/checkbox1/checkbox2/checkbox3/checkbox4/checkbox5/checkbox6/checkbox7) ▼ -----------------------    
    ## st.checkbox 사용
    ## 입력 변수명1: {기침:cough_check, 간헐적 경련:convulsion_check, 마비:paralysis_check, 무감각:insensitive_check, 통증:pain_check, 만지면 아픔: touch_pain_check}
    ## 입력 변수명2: {설사:diarrhea_check, 출혈:bleeding_check, 시력 저하:blurred_check, 호흡 곤란:breath_check, 현기증:dizziness_check}
    
    st.markdown("#### 증상 체크하기")
    
    col150, col151, col152, col153, col154, col155, col156, col157 = st.columns([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])  # col 나누기
    with col150:
        st.error("증상 체크")
    with col151:
        cough_check = st.checkbox("기침")
        convulsion_check = st.checkbox("간헐적 경련")
    with col152:
        paralysis_check = st.checkbox('마비')
        insensitive_check = st.checkbox('무감각')
    with col153:
        pain_check = st.checkbox('통증')
        touch_pain_check = st.checkbox('만지면 아픔')
    with col154:
        dysarthria_check = st.checkbox('말이 어눌해짐')
        swoon_check = st.checkbox('졸도')
    with col155:
        diarrhea_check = st.checkbox('설사')
        bleeding_check = st.checkbox('출혈')
    with col156:
        blurred_check = st.checkbox('시력 저하')
        breath_check = st.checkbox('호흡 곤란')
    with col157:
        dizziness_check = st.checkbox('현기증')
        
       
        
     # -------------------- ▼ 1-6그룹 중증 질환 여부, 중증 질환 판단(special_yn) col 구성 ▼ --------------------
    ## selectbox  사용(변수: special_yn)
    
    col160, col161, col162 = st.columns([0.1, 0.3, 0.4]) # col 나누기
    
    with col160:
        st.error("중증 질환 여부")
    with col161:
        special_yn = st.selectbox(' ',options=['중증 질환 선택', '중증 질환 아님', '중증 질환 예측'], label_visibility='collapsed')
        
        if special_yn == '중증 질환 선택':
            special_s = st.radio(' ',options=['심근경색', '복부손상', '뇌경색', '뇌출혈'], label_visibility='collapsed', horizontal=True)
    
    
            ## -------------------- ▼ 1-7그룹 중증 질환 선택 또는 예측 결과 표시 cols 구성 ▼ --------------------
                ## ---------------------------------------------------------------------------
        elif special_yn == "중증 질환 예측":

            patient_data = {
                "체온": [fever],
                "수축기 혈압": [high_blood],
                "이완기 혈압": [low_blood],
                "호흡 곤란": [int(breath_check)],
                "간헐성 경련": [int(convulsion_check)],
                "설사": [int(diarrhea_check)],
                "기침": [int(cough_check)],
                "출혈": [int(bleeding_check)],
                "통증": [int(pain_check)],
                "만지면 아프다": [int(touch_pain_check)],
                "무감각": [int(insensitive_check)],
                "마비": [int(paralysis_check)],
                "현기증": [int(dizziness_check)],
                "졸도": [int(swoon_check)],
                "말이 어눌해졌다": [int(dysarthria_check)],
                "시력이 흐려짐": [int(blurred_check)],
                "중증질환": [" "],

            }
            
            # AI 모델 중증질환 예측 함수 호출

    ## ---------------------------------------------------------------------------
            special_m = predict_disease(patient_data)
            
            st.markdown(f"### 예측된 중증 질환은 {special_m}입니다")
            st.write("중증 질환 예측은 뇌출혈, 뇌경색, 심근경색, 응급내시경 4가지만 분류됩니다.")
            st.write("이외의 중증 질환으로 판단될 경우, 직접 선택하세요")

        else:
            special_s = "중증 아님"
            st.write("")
    
            
            
#     ## -------------------- ▼ 1-8그룹 가용병원 표시 폼 지정 ▼ --------------------
#     st.markdown("중증 질환 선택")    
    
#     with st.form(key='tab1_first'):

#         ### 병원 조회 버튼 생성
#         if st.form_submit_button(label = "병원조회"):

#             #### 거리주소 -> 위도/경도 변환 함수 호출
#             lati, long = 

#             #### 인근 병원 찾기 함수 호출
#             hospital_list = 

#             #### 필요 병원 정보 추출 
#             display_column =  data.columns
#             display_df = st.dataframe(data[:10])
#             display_df.reset_index(drop=True, inplace=True)

#             #### 추출 병원 지도에 표시
#             with st.expander("인근 병원 리스트", expanded=True):
#                 st.dataframe(data[:10])
#                 m = folium.Map(location = [], zoom_start = 11)
                
                
#                 icon = folium.Icon(color = "red")
                
#                 folium.Marker(location = [).add_to(m)


#                 ###### folium을 활용하여 지도 그리기 (3일차 교재 branca 참조)


Overwriting dispatch_log.py


## 3) 프로토타입 확인하기

  * 아래 streamlit은 1회만 실행
  * app.py가 수정되면 프로토타입이 자동 변경됨
  * 새로고침을 통해 변경사항 확인

In [16]:
!streamlit run dispatch_log.py

^C


## [도전미션] 터널링을 통해 웹 서비스 하기

  * npm 설치: node js 사이트에서 18.15 버전 다운받아 설치
  * 재부팅 후 터미널에 "npm -v" 실행하여 설치여부 확인(9.5.0 버전이 나오면 설치 완료)
  * npm을 이용하여 localtunnel 설치(npm install -g localtunnel)
  * 스트림릿 실행(streamlit run app.py)
  * localtunnel 실행(lt --port [스트림릿 실행 포트] --subdomain [자신만의 웹주소] --print-requests)
  * 생성된 url 확인후 접속(your url is: https://[자신만의 웹주소].loca.lt)